# Trabajo de contexto con scraping

### Primero debemos hacer loigin en una plataforma

- prueba del funcionamiento de la key 

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')

MODEL = "gemini-2.0-flash"
openai = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta", api_key=api_key)

response = openai.chat.completions.create(
 model=MODEL,
 messages=[{"role": "user", "content": "¿Cuánto son 2 + 2?"}]
)

print(response.choices[0].message.content)

2 + 2 = 4



### Primeros pasos

analizamos una pagina de coches de segunda mano para hacer scraping de dicha pagina 

In [33]:
import requests
from bs4 import BeautifulSoup

# A class to represent a Webpage
# Code from: https://github.com/ed-donner/llm_engineering 

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [57]:

novak_url = "https://www.autofesa.com/"
novak_web = Website(novak_url)
print(novak_web.title)
print(novak_web.text[350:400])

Concesionario segunda mano Madrid | Venta de coches ocasión
oches
Tablas GANVAM
Coches km0
Ofertas especiales 


In [58]:
def user_prompt_for(website):
   
    user_prompt = (
        f"Analiza el siguiente sitio web de coches de segunda mano y genera un resumen en formato Markdown.\n"
        f"### Título del sitio:\n{website.title}\n\n"
        "### Instrucciones:\n"
        "- Resume el propósito principal del sitio (por ejemplo: compraventa, anuncios, concesionario, comparador, etc.).\n"
        "- Destaca la información clave sobre los vehículos ofrecidos (marcas, modelos, precios, kilometraje, ubicación, etc.).\n"
        "- Si hay noticias, reseñas o secciones informativas sobre automóviles, inclúyelas brevemente.\n"
        "- Usa encabezados y listas en Markdown para organizar la información.\n"
        "- Evita texto irrelevante como menús, publicidad o pies de página.\n\n"
        "### Contenido del sitio:\n"
        f"{website.text.strip()}\n"
    )
    return user_prompt

In [59]:
system_prompt = "Eres un asistente que analiza el contenido de un sitio web \
                    y proporciona un resumen breve, ignorando el texto que podría estar relacionado con la navegación. \
                    No añades ningún comentario inicial ni final. \
                    Respondes en markdown. Respondes en español."


def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [60]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [61]:
summarize(novak_url)

'# Concesionario Autofesa: Venta de coches de segunda mano y ocasión\n\nAutofesa es un concesionario online con presencia física en Madrid, especializado en la compra y venta de coches y motos de segunda mano y ocasión.\n\n## Información clave:\n\n*   **Propósito:** Compraventa de vehículos de segunda mano y ocasión (coches y motos). También compran vehículos usados.\n*   **Ubicación:** Madrid (Collado Villalba, Leganés, Aravaca).\n*   **Vehículos ofrecidos:**\n    *   Amplia variedad de marcas y modelos.\n    *   Kilometraje y precios variables.\n    *   Disponibilidad de utilitarios, berlinas, familiares, todoterrenos, deportivos, monovolúmenes, eléctricos, híbridos, furgonetas y motocicletas.\n    *   Opciones de coches KM 0, liquidación y ofertas especiales.\n*   **Servicios:**\n    *   Financiación.\n    *   Tasación online de vehículos para venta.\n    *   Garantía en vehículos.\n    *   Entrega a domicilio en toda la península (Ver condiciones).\n    *   Taller propio para revis

In [62]:
from IPython.display import Markdown, display

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [63]:
display_summary(novak_url)

# Concesionario Autofesa: Venta de coches de segunda mano y ocasión en Madrid

Autofesa es un concesionario online y físico especializado en la compraventa de coches y motos de segunda mano y ocasión. Ofrecen una amplia selección de vehículos revisados y garantizados, con opciones de financiación y la posibilidad de entregar el coche en toda la península.

## Información Clave:

*   **Propósito:** Venta de coches y motos de segunda mano y ocasión. También compran vehículos usados.
*   **Ubicación:** Madrid (Collado Villalba, Leganés, Aravaca).
*   **Vehículos Ofrecidos:**
    *   **Marcas:** Amplia variedad, incluyendo Abarth, Audi, BMW, Citroen, Ford, Jaguar, Mercedes-Benz, Opel, Peugeot, Renault, Seat, Volkswagen, Volvo, entre otras.
    *   **Modelos:** Diversos modelos de cada marca.
    *   **Tipos de Vehículos:** Utilitarios, berlinas, familiares, todoterrenos, pickups, deportivos, descapotables, monovolúmenes, eléctricos, híbridos, campers, furgonetas y motocicletas.
    *   **Kilometraje:** Variado, desde coches con pocos kilómetros hasta opciones con más de 300.000 km.
    *   **Precios:** Diferentes rangos de precios, incluyendo opciones económicas y vehículos de lujo.
*   **Servicios:**
    *   Venta de coches y motos de ocasión.
    *   Tasación y compra de vehículos usados.
    *   Financiación.
    *   Garantía y certificación de kilometraje.
    *   Taller propio para revisión y reparación de vehículos.
*   **Promociones:**
    *   Ofertas especiales en coches y motos.
    *   Liquidaciones.
    *   Coches con IVA deducible.
    *   Coches KM 0.
*   **Horario Especial:**
    *   El centro de Villalba abrirá el 12 de Octubre de 10 a 14:30 horas.
